# Init stuffs

### Input data

In [1]:
filename = "./input_1.txt"
test_filename = "./input_2.txt"

with open(file=filename) as f:
    data = f.read().splitlines()

with open(file=test_filename) as f:
    test_data = f.read().splitlines()

# Pt. 1

In [2]:
def init_structs(bricks_list: list[list[str]]) -> tuple[dict,tuple[int]]:
    bricks = {} # number: {start_position, final_position, supported_by, supports}
    max_x = 0
    max_y = 0
    max_z = 0

    for index, brick in enumerate(bricks_list):
        x1, y1, z1 = brick.split("~")[0].split(",")
        x2, y2, z2 = brick.split("~")[1].split(",")

        max_x = max(max_x, int(x2))
        max_y = max(max_y, int(y2))
        max_z = max(max_z, int(z2))

        brick_coords = [(x, y ,z) for x in range(int(x1), int(x2)+1) for y in range(int(y1), int(y2)+1) for z in range(int(z1), int(z2)+1)]
        bricks[index] = {"start_position": brick_coords, "final_position": [], "supported_by": [], "supports": []}

    return (bricks,(max_x, max_y, max_z))

def drop_blocks(bricks: list[list[str]], grid_dimensions: tuple[int]) -> tuple[dict, dict]:
    filled_grid = {(x, y): {"z": 0, "brick": -1} for x in range(grid_dimensions[0]+1) for y in range(grid_dimensions[1]+1)}

    falling_bricks = bricks.copy()

    while falling_bricks:
        min_z = min([min([item[2] for item in brick["start_position"]]) for brick in falling_bricks.values()])
        lowest_bricks = [k for k, v in falling_bricks.items() if v["start_position"][0][2] == min_z]

        for brick_id in lowest_bricks:
            brick = falling_bricks.pop(brick_id)
            bricks_below = [filled_grid[(x,y)] for x,y,_ in brick["start_position"]]
            max_z_below = max([z["z"] for z in bricks_below])
            supported_by = set([support["brick"] for support in bricks_below if support["z"]==max_z_below])
            dz = min_z - max_z_below - 1
            for coord in brick["start_position"]:
                bricks[brick_id]["final_position"].append((*coord[:2],coord[2]-dz))
                filled_grid[(coord[:2])] = {"z": coord[2]-dz, "brick":brick_id}
            bricks[brick_id]["supported_by"] = supported_by
            for support in supported_by:
                if support != -1:
                    bricks[support]["supports"].append(brick_id)
    return bricks, filled_grid

def find_removable(bricks: dict) -> list:
    removable = []
    for k, v in bricks.items():
        supporting = v["supports"]
        if all([len(bricks[higher_brick]["supported_by"]) > 1 for higher_brick in supporting]):
            removable.append(k)
    return removable


In [3]:
bricks_list = data
bricks, grid_dimensions = init_structs(bricks_list=bricks_list)
bricks, filled_grid = drop_blocks(bricks=bricks, grid_dimensions=grid_dimensions)
len(find_removable(bricks=bricks))

454

# Pt. 2

In [4]:
def remove_chain(bricks: list[list[str]]) -> int:
    chained_falls = 0
    for k, v in bricks.items():
        falling = set([k])
        actively_falling = [k]
        while actively_falling:
            currently_falling = actively_falling.pop()
            supporting = bricks[currently_falling]["supports"]
            for higher_brick in supporting:
                supporting_bricks = bricks[higher_brick]["supported_by"]
                if supporting_bricks.issubset(falling):
                    if higher_brick not in falling:
                        falling.add(higher_brick)
                        actively_falling.append(higher_brick)
        chained_falls += len(falling)-1
    return chained_falls

In [5]:
bricks_list = data
bricks, grid_dimensions = init_structs(bricks_list=bricks_list)
bricks, filled_grid = drop_blocks(bricks=bricks, grid_dimensions=grid_dimensions)
remove_chain(bricks=bricks)

74287